In [13]:
# %load GAIN_Linear_RMSE_cal_for_training_data_Heena.py
#!/usr/bin/env python

# In[3]:


# %load GAIN_Linear_RMSE_cal_for_training_data_Heena.py
#!/usr/bin/env python

# In[1]:


# %load openaps_gain.py
'''Main function for UCI letter and spam datasets.
'''

# Necessary packages
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import numpy as np
import pandas as pd

from data_loader import data_loader
from gain import gain
from utils import rmse_loss

 #Reading files from folder and making a list to loop through
import os
list_of_files_for_running_GAIN_LR_implementation = []
for participant in os.listdir(r'C:\Users\heena\Desktop\UW\UW_NN_Heena\HU_Done\UW_NN_Heena_improvised_codes\data'):
    if participant.endswith(".csv"): 
        list_of_files_for_running_GAIN_LR_implementation.append(participant[:-4])
print (list_of_files_for_running_GAIN_LR_implementation)

devicestatus_gain_rmse_each_participant = {}
devicestatus_linear_rmse_each_participant = {}

for participant in list_of_files_for_running_GAIN_LR_implementation:
    gain_parameters = {'batch_size': 16, 'hint_rate': 0.9, 'alpha': 10, 'iterations': 10000}
    
    # Impute missing data
    #Creating a list to keep rmse values for both gain and linear imputation for 10-70% i.e 
    gain_rmse_list_for_10to70_percent_missingness_for_each_participant = []
    linear_rmse_list_for_10to70_percent_missingness_for_each_participant =[]
    for i in np.arange(0.1, 0.8, 0.1):
        miss_rate = i
    # Load data and introduce missingness, we introduce missingness just once for both GAIN and linear both models
        ori_data_x, miss_data_x, data_m = data_loader(participant, miss_rate)

        gain_rmse_sum = 0
        #for each missing percentage gain is calculated 10 times below and then averaged 10 times for same misingness
        for _ in np.arange(10):
            imputed_data_x = gain(miss_data_x, gain_parameters)
            gain_rmse = rmse_loss(ori_data_x, imputed_data_x, data_m)
            gain_rmse_sum += gain_rmse
        gain_rmse_list_for_10to70_percent_missingness_for_each_participant.append(gain_rmse_sum/10)
        
        #Linear Regression imputation
        miss_data_x = pd.DataFrame(miss_data_x, columns=list('abcd'))

        miss_data_x['d'] = miss_data_x['d'].cumsum()

        miss_data_x.set_index('d', inplace=True)
        miss_data_x.interpolate(method='index', inplace=True, limit_direction='both')
        miss_data_x.reset_index(inplace=True)
        miss_data_x = miss_data_x[list('abcd')]
        miss_data_x = np.array(miss_data_x)

        linear_rmse = rmse_loss(ori_data_x, miss_data_x, data_m)

        linear_rmse_list_for_10to70_percent_missingness_for_each_participant.append(linear_rmse)
    devicestatus_gain_rmse_each_participant[participant] = gain_rmse_list_for_10to70_percent_missingness_for_each_participant
    devicestatus_linear_rmse_each_participant[participant] = linear_rmse_list_for_10to70_percent_missingness_for_each_participant
        
    #devicestatus_gain_rmse_each_participant[participant]=gain_rmse_list_for_10to70_percent_missingness_for_each_participant
    #we can't wait for all to finish, take note of every participant id where gain computation is completed
    #print(devicestatus_gain_rmse_each_participant)

print(devicestatus_gain_rmse_each_participant)
print(devicestatus_linear_rmse_each_participant)

['devicestatus_01352464']


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 10000/10000 [00:17<00:00, 574.26it/s]


{'devicestatus_01352464': [0.18653162098253118, 0.19494668717228078, 0.20564878531656183, 0.21708490806648184, 0.2139058368122826, 0.23567320498044625, 0.20870317683673512]}
{'devicestatus_01352464': [0.15190082055668364, 0.15226007191725974, 0.1551257957457253, 0.15668427430907472, 0.15680353725872181, 0.16065620307757514, 0.16627566688514525]}


In [16]:
print('GAIN results' , devicestatus_gain_rmse_each_participant)
print('Linear results' , devicestatus_linear_rmse_each_participant)


GAIN results {'devicestatus_01352464': [0.18653162098253118, 0.19494668717228078, 0.20564878531656183, 0.21708490806648184, 0.2139058368122826, 0.23567320498044625, 0.20870317683673512]}
Linear results {'devicestatus_01352464': [0.15190082055668364, 0.15226007191725974, 0.1551257957457253, 0.15668427430907472, 0.15680353725872181, 0.16065620307757514, 0.16627566688514525]}


In [18]:
from numpy import array
from numpy import sum


#We average gain measurement for each missing percentage over range of participants
gain_rmse_for_each_participant_in_training_set = []
for participant in devicestatus_gain_rmse_each_participant:
    gain_rmse_for_each_participant_in_training_set.append(devicestatus_gain_rmse_each_participant[participant])   
#print(gain_rmse_for_each_participant_in_training_set)


# In[27]:



data_for_overall_average_gain_rmse_over_entire_trainingset = array(gain_rmse_for_each_participant_in_training_set)
len(data_for_overall_average_gain_rmse_over_entire_trainingset)
overall_average_gain_rmse_over_entire_trainingset=sum(data_for_overall_average_gain_rmse_over_entire_trainingset,0)/len(data_for_overall_average_gain_rmse_over_entire_trainingset)


GAIN rmse imputation results for 10, 20, 30 ... 70% missingness [0.18653162 0.19494669 0.20564879 0.21708491 0.21390584 0.2356732
 0.20870318]


In [20]:
linear_rmse_for_each_participant_in_training_set = []
for participant in devicestatus_linear_rmse_each_participant:
    linear_rmse_for_each_participant_in_training_set.append(devicestatus_linear_rmse_each_participant[participant])
#print(linear_rmse_for_each_participant_in_training_set)



data_for_overall_average_linear_rmse_over_entire_trainingset = array(linear_rmse_for_each_participant_in_training_set)
len(data_for_overall_average_linear_rmse_over_entire_trainingset)
overall_average_linear_rmse_over_entire_trainingset=sum(data_for_overall_average_linear_rmse_over_entire_trainingset,0)/len(data_for_overall_average_linear_rmse_over_entire_trainingset)
print(overall_average_linear_rmse_over_entire_trainingset)



[0.15190082 0.15226007 0.1551258  0.15668427 0.15680354 0.1606562
 0.16627567]
Linear rmse imputation results for 10, 20, 30 ... 70% missingness [0.15190082 0.15226007 0.1551258  0.15668427 0.15680354 0.1606562
 0.16627567]


In [21]:
print('GAIN rmse imputation results for 10, 20, 30 ... 70% missingness', overall_average_gain_rmse_over_entire_trainingset )
print ('Linear rmse imputation results for 10, 20, 30 ... 70% missingness', overall_average_linear_rmse_over_entire_trainingset)

GAIN rmse imputation results for 10, 20, 30 ... 70% missingness [0.18653162 0.19494669 0.20564879 0.21708491 0.21390584 0.2356732
 0.20870318]
Linear rmse imputation results for 10, 20, 30 ... 70% missingness [0.15190082 0.15226007 0.1551258  0.15668427 0.15680354 0.1606562
 0.16627567]
